In [1]:
import json

import yaml

config = yaml.safe_load(open('config.yml'))
# config

In [2]:
from ollama import Client

client = Client(host='http://gpu02:11434')

model_name = "calebfahlgren/natural-functions"

In [13]:
import json
from helpers import has_function_call, extract_function_call_from_string
from tools import is_available_function, arxiv_func, wiki_func, search_func
from chat_history import ChatHistory

chat_history = ChatHistory(functions=config['functions'], history_limit=10)
# chat_history.get_messages()

while True:

    ### 1. Запрос пользователя

    # user_message = input("User: ")
    # user_message = "Find in arxiv papers about chain of thoughts"
    user_message = "Find in duckduckgo how reproduce segfault in php script"
    # user_message = "Find in wiki something about monkeys"

    # Reset chat command
    if user_message.strip() == "/reset":
        chat_history = ChatHistory(functions=config['functions'], history_limit=10)
        print("History reset completed!")
        continue

    # Skip empty messages from user
    if user_message.strip() == "":
        continue

    ### 2. Запрос передаётся на вход LLM

    chat_history.add_user_message(user_message)
    messages = chat_history.get_messages()
    model_response = client.chat(
        model=model_name, messages=messages,
        options={
            "num_predict": 1024,
            "num_ctx": 32768,
            "temperature": 0,
            "repetition_penalty": 1.1
        }
    )

    # Read model response and save it to chat history
    output = model_response['message']['content']
    chat_history.add_assistant_message(output)
    # print(output)

    ### 3. Решаем нужно ли вызвать и если да то какой

    # If response has function call
    function_call = None
    if has_function_call(output):
        # Extract functino call object
        function_call = extract_function_call_from_string(output)
        function_name = function_call['name']
        function_args = function_call['arguments']

        if not is_available_function(function_name):
            # print(f">>> {function_name} is not available")
            continue

        ### 4. Запрос в тул function call
   
        # print(f">>> {function_name} is available")
        function_response = ''
        if function_name == 'arxiv_func':
            function_response = arxiv_func().invoke(function_args['query'])
        if function_name == 'wiki_func':
            function_response = wiki_func().invoke(function_args['query'])
        if function_name == 'search_func':
            function_response = search_func().invoke(function_args['query'])

        # ---

        ### 5. Ответ тула function response

        # Add function response as user
        chat_history.add_user_message('FUNCTION RESPONSE: ' + json.dumps({"result": function_response}))


    ### 6. Интерпретация ответа

    messages = chat_history.get_messages()
    
    # print(messages)
    # break
    
    model_response = client.chat(
        model=model_name, messages=messages,
        options={
            "num_predict": 1024,
            "num_ctx": 32768,
            "temperature": 0,
            "repetition_penalty": 1.1
        }
    )
    
    # Read model response and save it to chat history
    output = model_response['message']['content']
    chat_history.add_assistant_message(output)
    print(output)

    # В начало

    break


Here are some resources that might help you reproduce the segmentation fault in your PHP script:

- [Troubleshooting PHP Segmentation Faults: Effective Debugging Tips - Atatus](https://www.atatus.com/blog/debugging-php/)
- [PHP :: Bug #17045 :: Apache 2.0.35 + PHP4.2.0 segfault on start](https://bugs.php.net/17045)
- [PHP :: Bug #64463 :: Segfault (For the moment, can't reproduce it)](https://bugs.php.net/bug.php?id=64463)
- [PHP :: Bug #79710 :: Reproducible segfault in error_handler during GC ...](https://bugs.php.net/79710)
